<a href="https://colab.research.google.com/github/jeramirez169/DataScience_SGS_Classification/blob/main/models/03_tranformer_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# Clonar tu repositorio desde GitHub
!git clone https://github.com/jeramirez169/DataScience_SGS_Classification.git
%cd DataScience_SGS_Classification

# Instalar dependencias necesarias
!pip install -q pandas numpy scikit-learn unidecode spacy imbalanced-learn transformers
!python -m spacy download es_core_news_lg


fatal: destination path 'DataScience_SGS_Classification' already exists and is not an empty directory.
/content/DataScience_SGS_Classification
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.0/568.0 MB 3.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [7]:
# Ir a la carpeta data
%cd data

# Descomprimir el archivo ZIP
!unzip "Dataset_SGS_clean.zip" -d .

# Regresar al directorio raíz del proyecto
%cd ..


/content/DataScience_SGS_Classification/data
Archive:  Dataset_SGS_clean.zip
replace ./Dataset_SGS_clean.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: /content/DataScience_SGS_Classification


In [3]:
!pip uninstall -y transformers accelerate datasets tokenizers sentencepiece huggingface_hub
!pip cache purge


Found existing installation: transformers 4.44.2
Uninstalling transformers-4.44.2:
  Successfully uninstalled transformers-4.44.2
Found existing installation: accelerate 0.33.0
Uninstalling accelerate-0.33.0:
  Successfully uninstalled accelerate-0.33.0
Found existing installation: datasets 3.0.1
Uninstalling datasets-3.0.1:
  Successfully uninstalled datasets-3.0.1
Found existing installation: tokenizers 0.19.1
Uninstalling tokenizers-0.19.1:
  Successfully uninstalled tokenizers-0.19.1
Found existing installation: sentencepiece 0.2.0
Uninstalling sentencepiece-0.2.0:
  Successfully uninstalled sentencepiece-0.2.0
Found existing installation: huggingface-hub 0.36.0
Uninstalling huggingface-hub-0.36.0:
  Successfully uninstalled huggingface-hub-0.36.0
Files removed: 42


In [4]:
!pip install transformers==4.44.2 accelerate==0.33.0 datasets==3.0.1 tokenizers==0.19.1 sentencepiece==0.2.0


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 53.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.1/315.1 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 34.0 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer
)
import torch
import os


RuntimeError: Failed to import transformers.trainer because of the following error (look up to see its traceback):
No module named 'transformers.modeling_layers'

In [ ]:
# Clonar tu repositorio desde GitHub
!git clone https://github.com/jeramirez169/DataScience_SGS_Classification.git
%cd DataScience_SGS_Classification

# Instalar y actualizar dependencias necesarias
!pip install -q --upgrade pandas numpy scikit-learn unidecode spacy imbalanced-learn transformers peft accelerate
!python -m spacy download es_core_news_lg

In [4]:
ruta = "data/Dataset_SGS_clean.csv"
df = pd.read_csv(ruta, encoding="utf-8")

df = df[['Oficina', 'texto_truncado']].rename(columns={
    'Oficina': 'label_text',
    'texto_truncado': 'text'
})

label2id = {label: idx for idx, label in enumerate(sorted(df['label_text'].unique()))}
id2label = {v: k for k, v in label2id.items()}
df['label'] = df['label_text'].map(label2id)

print("Etiquetas y códigos:", label2id)
print("Total de muestras:", len(df))




Etiquetas y códigos: {'Biblioteca': 0, 'Desarrollo de sistemas': 1, 'Infraestructura': 2, 'Multimedios': 3, 'Operaciones': 4, 'Servicios y proyectos': 5}
Total de muestras: 69746


In [5]:
df.shape

NameError: name 'df' is not defined

In [6]:
df_train, df_test = train_test_split(
    df,
    test_size=0.2,
    stratify=df['label'],
    random_state=42
)

print(f"Tamaño entrenamiento: {len(df_train)}")
print(f"Tamaño prueba: {len(df_test)}")



Tamaño entrenamiento: 55796
Tamaño prueba: 13950


In [7]:
#Tokenización
MODEL_NAME = "dccuchile/bert-base-spanish-wwm-uncased"
MAX_LEN = 256

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

train_ds = Dataset.from_pandas(df_train)
test_ds = Dataset.from_pandas(df_test)

def tokenize(batch):
    return tokenizer(
        batch["text"],
        truncation=True,
        padding="max_length",
        max_length=MAX_LEN
    )

train_ds = train_ds.map(tokenize, batched=True)
test_ds = test_ds.map(tokenize, batched=True)

cols = ["input_ids", "attention_mask", "label"]
train_ds.set_format("torch", columns=cols)
test_ds.set_format("torch", columns=cols)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/55796 [00:00<?, ? examples/s]

Map:   0%|          | 0/13950 [00:00<?, ? examples/s]

In [8]:
#Cargar modelo BETO
NUM_LABELS = len(label2id)

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=NUM_LABELS,
    id2label=id2label,
    label2id=label2id
)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
#Calcular pesos de clase (para desbalance)
counts = df_train['label'].value_counts().sort_index().values
class_weights = torch.tensor(
    (counts.sum() / (NUM_LABELS * counts)),
    dtype=torch.float
)
print("Pesos de clase:", class_weights)


Pesos de clase: tensor([5.4993, 1.7743, 1.6108, 0.3759, 0.6157, 2.8622])


In [10]:
#Función de perdida
def custom_loss(model, inputs, return_outputs=False):
    labels = inputs.pop("labels")
    outputs = model(**inputs)
    logits = outputs.logits
    loss_fn = torch.nn.CrossEntropyLoss(weight=class_weights.to(logits.device))
    loss = loss_fn(logits, labels)
    return (loss, outputs) if return_outputs else loss


In [11]:
#Métricas
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=-1)
    macro_f1 = f1_score(labels, preds, average="macro")
    weighted_f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {
        "accuracy": acc,
        "macro_f1": macro_f1,
        "weighted_f1": weighted_f1
    }

In [4]:
args = TrainingArguments(
    output_dir="./beto_sgs_80_20",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=5,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="macro_f1",
    logging_steps=100,
    warmup_ratio=0.1,
    weight_decay=0.01,
    seed=42
)



/usr/local/lib/python3.12/dist-packages/transformers/training_args.py:1559: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
#Entrenamiento
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.compute_loss = custom_loss
trainer.train()



In [ ]:
#Evaluación
metrics_test = trainer.evaluate(test_ds)
print("\nResultados en conjunto de prueba (20%):")
for k, v in metrics_test.items():
    print(f"{k}: {v:.4f}")



In [ ]:
#Matriz de confusión
preds = trainer.predict(test_ds)
y_true = preds.label_ids
y_pred = preds.predictions.argmax(-1)

cm = confusion_matrix(y_true, y_pred, normalize='true')

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, cmap="Blues", fmt=".2f",
            xticklabels=id2label.values(),
            yticklabels=id2label.values())
plt.xlabel("Predicción")
plt.ylabel("Etiqueta real")
plt.title("Matriz de Confusión - BETO SGS (80/20)")
plt.show()

